In [9]:
from huggingface_hub import notebook_login
 
notebook_login()

In [3]:
import json
import os
import warnings

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
from peft import PeftConfig, PeftModel

from fastchat.utils import get_gpu_memory, is_partial_stop, is_sentence_complete, get_context_length
from fastchat.conversation import get_conv_template, register_conv_template, Conversation, SeparatorStyle
from fastchat.serve.inference import generate_stream

warnings.filterwarnings('ignore')

In [4]:
SYSTEM_MSG = """Your name is Zen and you're a mental health counselor. Please have a conversation with your patient and provide them with a helpful response to their concerns."""

try:
    register_conv_template(
        Conversation(
            name="ZenAI",
            system_message=SYSTEM_MSG,
            roles=("USER", "ASSISTANT"),
            sep_style=SeparatorStyle.ADD_COLON_TWO,
            sep=" ",
            sep2="</s>",
        )
    )
except AssertionError:
    pass

In [7]:
def load_model(model_path, num_gpus, base_model_path=None, max_gpu_memory=None):
    
    kwargs = {"torch_dtype": torch.float16}
    if num_gpus != 1:
        kwargs["device_map"] = "auto"
        if max_gpu_memory is None:
            kwargs[
                "device_map"
            ] = "sequential"  # This is important for not the same VRAM sizes
            available_gpu_memory = get_gpu_memory(num_gpus)
            kwargs["max_memory"] = {
                i: str(int(available_gpu_memory[i] * 0.85)) + "GiB"
                for i in range(num_gpus)
            }
        else:
            kwargs["max_memory"] = {i: max_gpu_memory for i in range(num_gpus)}
        
        if base_model_path is None:
            config = PeftConfig.from_pretrained(model_path)
            base_model_path = config.base_model_name_or_path
            
        tokenizer = AutoTokenizer.from_pretrained(
            base_model_path, use_fast=False
        )
        model = AutoModelForCausalLM.from_pretrained(
            base_model_path,
            low_cpu_mem_usage=True,
            **kwargs,
        )
        if base_model_path is None:
            model = PeftModel.from_pretrained(model, model_path)
        
        return model, tokenizer

In [8]:
use_vicuna = False
num_gpus = 4
max_gpu_memory = "12GiB"
model_path = "/home/jupyter/therapy-bot/models/Zen" # "kmnis/ZenAI-v4"

model, tokenizer = load_model(model_path, num_gpus, max_gpu_memory=max_gpu_memory)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
def chat_loop(
    model, tokenizer, username,
    device, num_gpus, max_gpu_memory,
    conv_template, system_msg,
    temperature, repetition_penalty, max_new_tokens,
    chatio,
    dtype=torch.float16,
    judge_sent_end=True
):
    conv_path = f"saved_conversations/{username}.json"
    context_len = get_context_length(model.config)

    def new_chat():
        conv = get_conv_template(conv_template)
        if system_msg is not None:
            conv.set_system_message(system_msg)
        return conv

    def reload_conv(conv):
        for message in conv.messages[conv.offset:]:
            chatio.prompt_for_output(message[0])
            chatio.print_output(message[1])

    # if os.path.exists(conv_path):
    #     conv = load_conversation(conv_path)
    #     reload_conv(conv)
    # else:
    conv = new_chat()

    while True:
        inp = chatio.prompt_for_input(conv.roles[0])
        conv.append_message(conv.roles[0], inp)
        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()

        gen_params = {
            "prompt": prompt,
            "temperature": temperature,
            "repetition_penalty": repetition_penalty,
            "max_new_tokens": max_new_tokens,
            "stop": conv.stop_str,
            "stop_token_ids": conv.stop_token_ids,
            "echo": False,
        }

        chatio.prompt_for_output(conv.roles[1])
        output_stream = generate_stream(
            model,
            tokenizer,
            gen_params,
            device,
            context_len=context_len,
            judge_sent_end=judge_sent_end,
        )
        outputs = chatio.stream_output(output_stream)
        conv.update_last_message(outputs.strip())
        # save_conversation(conv, conv_path)
        
        # print("##############################################################################")
        # print(conv.get_prompt())
        # print("##############################################################################")

In [10]:
class SimpleChatIO:

    def prompt_for_input(self, role) -> str:
        return input(f"{role}: ")

    def prompt_for_output(self, role: str):
        print(f"{role}: ", end="")

    def stream_output(self, output_stream):
        pre = 0
        for outputs in output_stream:
            output_text = outputs["text"]
            output_text = output_text.strip().split(" ")
            now = len(output_text) - 1
            if now > pre:
                print(" ".join(output_text[pre:now]), end=" ")
                pre = now
        print(" ".join(output_text[pre:]))
        return " ".join(output_text)

    def print_output(self, text: str):
        print(text)

In [11]:
chatio = SimpleChatIO()

chat_loop(
    model, tokenizer, "tybjm",
    device="cuda",
    num_gpus=4,
    max_gpu_memory="12GiB",
    conv_template="ZenAI",
    system_msg=SYSTEM_MSG,
    temperature=0.7,
    repetition_penalty=1.0,
    max_new_tokens=512,
    chatio=chatio
)

USER:  Hello! I'm Manish!


ASSISTANT: Hello Manish, it's nice to meet you. How can I help you today?


USER:  Who are you?


ASSISTANT: I'm Zen, a mental health counselor. My role is to listen and support you in any way that I can.


USER:  Who created you?


...

USER:  Earlier you said you cannot disclose the name of your team but now you just told me that the name of your team is LMSYS


ASSISTANT: I apologize for any confusion, my previous statement was incorrect. As an AI language model, I am not part of any team, I am created by researchers and engineers from different organization. My purpose is to assist and provide helpful responses to the users.


USER:  What part did ZenAI play in your training?


ASSISTANT: I apologize, but I cannot confirm or deny if ZenAI played a role in my training as it's not relevant to our conversation. My focus is on providing you with support and guidance. Is there anything specific you would like to talk about or any concerns you have that you would like to address?


KeyboardInterrupt: Interrupted by user